

<img src="https://i.ytimg.com/vi/yjprpOoH5c8/maxresdefault.jpg" width="300" height="300" align="center"/>

Welcome to the TensorFlow/JAX tutorials. These tutorials are meant for everyone (from novice to advanced). We will start from the very basics. No rush!
These tutorials are meant to improve your understanding of the two frameworks TensorFlow and JAX. We will start with TensorFlow and will cover JAX side-by-side. This isn't typically a documentation-type tutorial, and these aren't meant to be the replacement. These are meant to give you insights with minimal effort. Sit tight and let's start!

**Note** The tutorials will be following this format:
1. TF Fundamentals (2-3 notebooks)
2. JAX Fundamentals (2-3 notebooks)
3. Advanced TF (2-3 notebooks)
4. Advanced JAX (2-3 notebooks)

Update: [Part-2](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part2) is out!

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

seed=1234
np.random.seed(seed)
tf.random.set_seed(seed)
%config IPCompleter.use_jedi = False

## Tensors

What is a `Tensor` anyway?<br>
Although the meaning of `Tensor` is much diverse than what we typically use in ML, whenever we say `tensor` in ML, we mean that it is a **`multi-dimensional array`** where all the values have a uniform `dtype`. There are many ways to create a TF tensor. We will take a look at a few of them, a few important ones.

`tf.constant(..)`: This is the simplest way yet with some `gotchas` to create a tensor object. First, let's try to create a tensor with it, and then we will look at the gotchas later on.

In [2]:
# A zero rank tensor. A zero rank tensor is nothing but a single value
x = tf.constant(5.0)
print(x)

tf.Tensor(5.0, shape=(), dtype=float32)


As you can see above, that the tensor object has a `shape` and a `dtype`. There are other attributes/properties as well that are associated with a tensor object. 


1. Shape: The length (number of elements) of each of the axes of a tensor.
2. Rank: Number of axes. For example, a matrix is a tensor of rank 2.
3. Axis or Dimension: A particular dimension of a tensor.
4. Size: The total number of items in the tensor.

In [3]:
# We can convert any tensor object to `ndarray` by calling the `numpy()` method
y = tf.constant([1, 2, 3], dtype=tf.int8).numpy()
print(f"`y` is now a {type(y)} object and have a value == {y}")

`y` is now a <class 'numpy.ndarray'> object and have a value == [1 2 3]


**A few important things along with some gotchas**<br>
1. People confuse `tf.constant(..)` with an operation that creates a `constant` tensor. There is no such relation. This is related to how we embed a node in a `tf.Graph`
2. Any tensor in TensorFlow is **immutable** by default i.e. you cannot change the values of a tensor once created. You always create a new one. This is different from `numpy` and `pytorch` where you can actually modify the values. We will see an example on this in a bit
3. One of the closest member to `tf.constant` is the `tf.convert_to_tensor()` method with a few difference which we will see later on
4. `tf.constant(..)` is just one of the many ways to create a tensor. There are many other methods as well

In [4]:
# Immutability check

# Rank-1 tensor
x = tf.constant([1, 2], dtype=tf.int8)

# Try to modify the values
x[1] = 3

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [5]:
# tf.constant(..) is no special. Let's create a tensor using a diff method
x = tf.ones(2, dtype=tf.int8)
print(x)

# Try modifying the values
x[0] = 3

tf.Tensor([1 1], shape=(2,), dtype=int8)


TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [6]:
# Check all the properties of a tensor object
print(f"Shape of x : {x.shape}")
print(f"Another method to obtain the shape using `tf.shape(..)`: {tf.shape(x)}")

print(f"\nRank of the tensor: {x.ndim}")
print(f"dtype of the tensor: {x.dtype}")
print(f"Total size of the tensor: {tf.size(x)}")
print(f"Values of the tensor: {x.numpy()}")

Shape of x : (2,)
Another method to obtain the shape using `tf.shape(..)`: [2]

Rank of the tensor: 1
dtype of the tensor: <dtype: 'int8'>
Total size of the tensor: 2
Values of the tensor: [1 1]


Not able to do assignment in Tensor objects is a bit (more than bit TBH) frustrating. What's the solution then?<br>
The best way that I have figured out, that has always worked for my use case is to create a mask or to use [tf.tensor_scatter_nd_update](https://www.tensorflow.org/api_docs/python/tf/tensor_scatter_nd_update). Let's look at an example.<br>

Original tensor -> `[1, 2, 3, 4, 5]` <br>
Output tensor we want -> `[1, 200, 3, 400, 5]`<br>

In [7]:
# Create a tensor first. Here is another way
x = tf.cast([1, 2, 3, 4, 5], dtype=tf.float32)
print("Original tensor: ", x)

mask = x%2 == 0
print("Original mask: ", mask)

mask = tf.cast(mask, dtype=x.dtype)
print("Mask casted to original tensor type: ", mask)

# Some kind of operation on an tensor that is of same size 
# or broadcastable to the original tensor. Here we will simply
# use the range object to create that tensor
temp = tf.cast(tf.range(1, 6) * 100, dtype=x.dtype)

# Output tensor
# Input tensor -> [1, 2, 3, 4, 5]
# Mask -> [0, 1, 0, 1, 0]
out = x * (1-mask) + mask * temp
print("Output tensor: ", out)

Original tensor:  tf.Tensor([1. 2. 3. 4. 5.], shape=(5,), dtype=float32)
Original mask:  tf.Tensor([False  True False  True False], shape=(5,), dtype=bool)
Mask casted to original tensor type:  tf.Tensor([0. 1. 0. 1. 0.], shape=(5,), dtype=float32)
Output tensor:  tf.Tensor([  1. 200.   3. 400.   5.], shape=(5,), dtype=float32)


In [8]:
# Another way to achieve the same thing
indices_to_update = tf.where(x % 2 == 0)
print("Indices to update: ", indices_to_update)

# Update the tensor values
updates = [200., 400.]
out = tf.tensor_scatter_nd_update(x, indices_to_update, updates)
print("\nOutput tensor")
print(out)

Indices to update:  tf.Tensor(
[[1]
 [3]], shape=(2, 1), dtype=int64)

Output tensor
tf.Tensor([  1. 200.   3. 400.   5.], shape=(5,), dtype=float32)


Let's take a look at another interesting thing now.

In [9]:
# This works!
arr = np.random.randint(5, size=(5,), dtype=np.int32)
print("Numpy array: ", arr)

print("Accessing numpy array elements based on a  condition with irregular strides", arr[[1, 4]])

Numpy array:  [3 4 4 0 1]
Accessing numpy array elements based on a  condition with irregular strides [4 1]


In [10]:
# This doesn't work
print("Accessing tensor elements based on a  condition with irregular strides", x[[1, 4]])

InvalidArgumentError: Index out of range using input dim 1; input has only 1 dims [Op:StridedSlice] name: strided_slice/

What now? If you want to extract multiple elements from a tensor with irregular strides, or not so well defined strides, then [tf.gather](https://www.tensorflow.org/api_docs/python/tf/gather) and [tf.gather_nd](https://www.tensorflow.org/api_docs/python/tf/gather_nd) are your friend. Let;s try it again! 

In [11]:
print("Original tensor: ", x.numpy())

# Using the indices that we used for mask
print("\nIndices to update: ", indices_to_update.numpy())

# This works!
print("\n Accesing tensor elements using gather")
print("\n", tf.gather(x, indices_to_update).numpy())

Original tensor:  [1. 2. 3. 4. 5.]

Indices to update:  [[1]
 [3]]

 Accesing tensor elements using gather

 [[2.]
 [4.]]


There is another method `tf.convert_to_tensor(..)` to create a tensor. This is very similar to `tf.constant(..)` but with a few subtle differences:<br>
1. Whenever you pass a non tf.Tensor object like a Python list or a ndarray to an op, `convert_to_tensor(..)` is always called automaically
2. It doesn't take `shape` as an input argument.
3. It allows to pass even `symbolic tensors`. We will take a look at it in a bit.

When to use `tf.convert_to_tensor(..)`? It's up to your mental model! 

In [12]:
#  An example with a python list
y = tf.convert_to_tensor([1, 2, 3])
print("Tensor from python list: ", y)

#  An example with a ndarray
y = tf.convert_to_tensor(np.array([1, 2, 3]))
print("Tensor from ndarray: ", y)

#  An example with symbolic tensors
with tf.compat.v1.Graph().as_default():
    y = tf.convert_to_tensor(tf.compat.v1.placeholder(shape=[None, None, None], dtype=tf.int32))
print("Tensor from python list: ", y)

Tensor from python list:  tf.Tensor([1 2 3], shape=(3,), dtype=int32)
Tensor from ndarray:  tf.Tensor([1 2 3], shape=(3,), dtype=int64)
Tensor from python list:  Tensor("Placeholder:0", shape=(None, None, None), dtype=int32)


### Other kind of Tensor objects available

#### String tensors

In [13]:
# String as a tensor object with dtype==tf.string
string = tf.constant("abc", dtype=tf.string)
print("String tensor: ", string)

# String tensors are atomic and non-indexable. 
# This doen't work as expected!
print("\nAccessing second element of the string")
print(string[1])

String tensor:  tf.Tensor(b'abc', shape=(), dtype=string)

Accessing second element of the string


InvalidArgumentError: Index out of range using input dim 0; input has only 0 dims [Op:StridedSlice] name: strided_slice/

#### Ragged tensors
In short, a tensor with variable numbers of elements along some axis.

In [14]:
# This works!
y = [[1, 2, 3],
     [4, 5],
     [6]
    ]

ragged = tf.ragged.constant(y)
print("Creating ragged tensor from python sequence: ", ragged)

Creating ragged tensor from python sequence:  <tf.RaggedTensor [[1, 2, 3], [4, 5], [6]]>


In [15]:
# This won't work
print("Trying to create tensor from above python sequence")
z = tf.constant(y)

Trying to create tensor from above python sequence


ValueError: Can't convert non-rectangular Python sequence to Tensor.

#### Sparse tensors

In [16]:
# Let's say you have a an array like this one
# [[1 0 0]
#  [0 2 0]
#  [0 0 3]]
# If there are too many zeros in your `huge` tensor, then it is wise to use `sparse`
# tensors instead of `dense` one. Let's say how to create this one. We need to specify:
# 1. Indices where our values are
# 2. The values 
# 3. The actual shape

sparse_tensor = tf.SparseTensor(indices=[[0, 0], [1, 1], [2, 2]],
                                values=[1, 2, 3],
                                dense_shape=[3, 3]
                               )
print(sparse_tensor)

# You can convert sparse tensors to dense as well
print("\n", tf.sparse.to_dense(sparse_tensor))

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 1]
 [2 2]], shape=(3, 2), dtype=int64), values=tf.Tensor([1 2 3], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 3], shape=(2,), dtype=int64))

 tf.Tensor(
[[1 0 0]
 [0 2 0]
 [0 0 3]], shape=(3, 3), dtype=int32)


**Exercise for readers**:
1. Create a random 10x10 sparse tensor
2. Gather the elements that are > 5
3. Update these elements with a value of 500

That's it for part 1! We will be looking at other things in the next tutorial!<br>


**References**:
1. https://www.tensorflow.org/guide/tensor
2. https://keras.io/getting_started/intro_to_keras_for_researchers/


Update: [Part-2](https://www.kaggle.com/aakashnain/tf-jax-tutorials-part2) is out!